In [204]:
import numpy as np
import pandapower as pp
import pandapower.networks as pn
from pandapower import runpp, runopp
import pandas as pd
import pyomo.environ as pyo
import copy

In [205]:
class Component: # Basic component in power system optimization model

    def __init__(self, **kwargs):
        self.baseMVA = 100
        pass

    def create_parameters(self):
        pass

    def update_timeseries_parameters(self, date):
        pass

    def create_variables(self):
        pass

    def create_expressions(self):
        pass

In [206]:
class Bus(Component):

    def __init__(self, name: str, data: pd.Series, load_data: pd.Series, shunt_data: pd.Series):
        super().__init__()

        self.name = name
        self.type = data["type"]
        self.vbase = data["vn_kv"]
        self.vmax = data["max_vm_pu"]
        self.vmin = data["min_vm_pu"]
        self.pD = load_data["p_mw"] / self.baseMVA
        self.qD = load_data["q_mvar"] / self.baseMVA
        self.qShunt = shunt_data["q_mvar"] / self.baseMVA
        self.slack_cost = None # Generation cost for external grid (for slack bus only)

        self.generators = []
        self.in_lines = []
        self.out_lines = []
        
    @classmethod
    def from_series(cls, data: pd.Series, load_data: pd.Series, shunt_data: pd.Series):
        return cls(data.name, data, load_data, shunt_data)

class Generator(Component):

    def __init__(self, name: str, data: pd.Series, cost_data: pd.Series):
        super().__init__()

        self.name = name
        self.bus_ID = data["bus"]
        self.pmin = data["min_p_mw"] / self.baseMVA
        self.pmax = data["max_p_mw"] / self.baseMVA
        self.qmin = data["min_q_mvar"] / self.baseMVA
        self.qmax = data["max_q_mvar"] / self.baseMVA
        self.cost = cost_data[["cp0_eur", "cp1_eur_per_mw", "cp2_eur_per_mw2"]].astype(np.float64).values.flatten()
        
        self.bus = None
        
    @classmethod
    def from_series(cls, data: pd.Series, cost_data: pd.Series):
        return cls(data.name, data, cost_data)

    def link_bus(self, buses: dict):
        # Link node to resource
        self.bus = buses[self.bus_ID]
        # Link resource to node
        buses[self.bus_ID].generators.append(self)

class Line(Component):

    def __init__(self, name: str, data: pd.Series):
        super().__init__()

        self.name = name
        self.from_bus_ID = data["from_bus"]
        self.to_bus_ID = data["to_bus"]
        # self.r_ohm = data["length_km"] * data["r_ohm_per_km"]
        # self.x_ohm = data["length_km"] * data["x_ohm_per_km"]
        # self.c_nf = data["length_km"] * data["c_nf_per_km"] # Line shunt admittance
        self.imax = data["max_i_ka"]

        # # Per unit line impedance / admittance values
        # self.z, self.r, self.x, self.y, self.g, self.b = None, None, None, None, None, None

        self.from_bus = None
        self.to_bus = None

    # def calculate_admittance(self):
    #     # Calculate base impedance (ohms)
    #     vbase = self.from_bus.vbase * 1e3 # convert kV to V
    #     sbase = self.baseMVA * 1e6 # convert MVA to VA
    #     zbase = vbase**2 / sbase

    #     assert abs(self.from_bus.vbase - self.to_bus.vbase) < 1e-3, "Voltage base mismatch across line ends."

    #     # Convert r and x to per unit
    #     self.r = self.r_ohm / zbase
    #     self.x = self.x_ohm / zbase
    #     self.z = complex(self.r, self.x) # Impedance

    #     # Calculate y, g and b
    #     self.y = 1 / self.z if abs(self.z) > 0 else 0 # Admittance
    #     self.g, self.b = np.real(self.y), np.imag(self.y)

    @classmethod
    def from_series(cls, data: pd.Series):
        return cls(data.name, data)

    def link_buses(self, buses: dict):
        # Link buses to line
        self.from_bus = buses[self.from_bus_ID]
        self.to_bus = buses[self.to_bus_ID]
        # Link line to nodes
        buses[self.from_bus_ID].out_lines.append(self)
        buses[self.to_bus_ID].in_lines.append(self)
        

In [207]:
class Model:

    def __init__(self, network, settings=None):

        # PandaPower network and model settings
        self.net = network
        self.settings = settings
        self.baseMVA = network.sn_mva
        
        # Power system components
        self.buses = {}
        self.generators = {}
        self.lines = {}

    @property
    def components(self):
        return list(self.buses.values() + self.generators.values() + self.lines.values())

    def create_system(self):

        # Run power flow
        runpp(self.net)

        # Pre-process data
        net = copy.copy(self.net)
        # Add loads to all buses
        net.load = net.load.set_index("bus").reindex(net.bus.index)
        net.load[["p_mw", "q_mvar"]] = net.load[["p_mw", "q_mvar"]].fillna(0)
        # Add shunts to all busees
        net.shunt = net.shunt.set_index("bus").reindex(net.bus.index)
        net.shunt["q_mvar"] = net.shunt["q_mvar"].fillna(0)

        # Get generator / slack bus cost data
        gen_cost = net.poly_cost.loc[net.poly_cost["et"] == "gen"].set_index("element")
        slack_cost = net.poly_cost.loc[net.poly_cost["et"] == "ext_grid"].set_index("element")
        
        # Create buses, generators, and line instances from network data
        for bus in net.bus.index:
            self.buses[bus] = Bus.from_series(net.bus.loc[bus], net.load.loc[bus], net.shunt.loc[bus])
        
        for gen in net.gen.index:
            self.generators[gen] = Generator.from_series(net.gen.loc[gen], net.poly_cost.loc[gen])
            self.generators[gen].link_bus(self.buses) # Link generator to node
            
        for line in net.line.index:
            self.lines[line] = Line.from_series(net.line.loc[line])
            self.lines[line].link_buses(self.buses) # Link line to nodes

        # Create admittance matrix
        Y_int = np.array(net._ppc["internal"]["Ybus"].todense())
        ext_to_int = net._pd2ppc_lookups["bus"]
        Y = np.zeros(Y_int.shape, dtype=np.complex128)
        for i in net.bus.index:
            for j in net.bus.index:
                Y[i,j] = Y_int[ext_to_int[i], ext_to_int[j]]
        self.Y = Y
        
        # Split real and imaginary components
        self.G = np.real(self.Y)
        self.B = np.imag(self.Y)

        # Determine slack bus
        self.slack_bus = self.buses[int(self.net.ext_grid.bus.values[0])]
        self.slack_bus.slack_cost = slack_cost[["cp0_eur", "cp1_eur_per_mw", "cp2_eur_per_mw2"]].values.flatten()

    def build_model(self):
        pass

    def solve_model(self):
        pass


class SDP_AC_OPF(Model):
    pass

In [246]:
class Poly_AC_OPF(Model):

    def __init__(self, network, settings=None):
        super().__init__(network, settings)

        # Pyomo Model
        self.model = None

    def build_model(self):

        # Instantiate model
        model = pyo.ConcreteModel()
        self.model = model
        
        # Create sets
        model.buses = pyo.Set(initialize=list(self.buses.keys()))
        model.generators = pyo.Set(initialize=list(self.generators.keys()))
        model.lines = pyo.Set(initialize=list(self.lines.keys()))
        model.slack = pyo.Set(initialize=[self.slack_bus.name])
        
        # Create parameters
        model.P_load = pyo.Param(model.buses, initialize={i: bus.pD for i, bus in self.buses.items()}, mutable=True)
        model.Q_load = pyo.Param(model.buses, initialize={i: bus.qD for i, bus in self.buses.items()}, mutable=True)
        
        # Create variables
        
        # Bus voltage
        model.V_re = pyo.Var(model.buses, within=pyo.Reals, initialize=1.0)
        model.V_im = pyo.Var(model.buses, within=pyo.Reals, initialize=0.0)
        
        # Generation
        model.P_gen = pyo.Var(model.generators, within=pyo.Reals, initialize=0.5)
        model.Q_gen = pyo.Var(model.generators, within=pyo.Reals, initialize=0.0)

        # External grid (slack bus) generation
        model.P_slack = pyo.Var(model.slack, within=pyo.Reals, initialize=0.5)
        model.Q_slack = pyo.Var(model.slack, within=pyo.Reals, initialize=0.0)

        # Write constraints
        
        # Power flow balance constraints
        def P_balance_rule(m, k):
            bus = self.buses[k]
            P_gen = sum(m.P_gen[g.name] for g in bus.generators)
            if k in model.slack:
                P_gen += m.P_slack[k]
            P_flow = m.V_re[k] * sum(self.G[k,i] * m.V_re[i] - self.B[k,i] * m.V_im[i] for i in m.buses) \
                   + m.V_im[k] * sum(self.B[k,i] * m.V_re[i] + self.G[k,i] * m.V_im[i] for i in m.buses)
            return P_gen == P_flow + m.P_load[k]
        model.P_balance = pyo.Constraint(model.buses, rule=P_balance_rule)
        
        def Q_balance_rule(m, k):
            bus = self.buses[k]
            Q_gen = sum(m.Q_gen[g.name] for g in bus.generators)
            if k in model.slack:
                Q_gen += m.Q_slack[k]
            Q_flow = m.V_re[k] * sum(-self.B[k,i] * m.V_re[i] - self.G[k,i] * m.V_im[i] for i in m.buses) \
                   + m.V_im[k] * sum(self.G[k,i] * m.V_re[i] - self.B[k,i] * m.V_im[i] for i in m.buses)
            return Q_gen == Q_flow + m.Q_load[k]
        model.Q_balance = pyo.Constraint(model.buses, rule=Q_balance_rule)
        
        # Voltage magnitude constraints
        def V_mag_rule(m, k):
            bus = self.buses[k]
            return pyo.inequality(bus.vmin**2, m.V_re[k]**2 + m.V_im[k]**2, bus.vmax**2)
        model.V_mag = pyo.Constraint(model.buses, rule=V_mag_rule)

        # Fix slack bus voltage and voltage angle
        for slack in model.slack:
            model.V_re[slack].fix(1.0)
            model.V_im[slack].fix(0.0)
        
        # Generator limits
        def P_gen_limits_rule(m, g):
            gen = self.generators[g]
            return pyo.inequality(gen.pmin, m.P_gen[g], gen.pmax)
        model.P_gen_limits = pyo.Constraint(model.generators, rule=P_gen_limits_rule)
        
        def Q_gen_limits_rule(m, g):
            gen = self.generators[g]
            return pyo.inequality(gen.qmin, m.Q_gen[g], gen.qmax)
        model.Q_gen_limits = pyo.Constraint(model.generators, rule=Q_gen_limits_rule)

        # Set objective
        def objective_rule(m):
            cost = sum(gen.cost[0] + gen.cost[1] * (gen.baseMVA * m.P_gen[i]) + gen.cost[2] * (gen.baseMVA * m.P_gen[i])**2 for i, gen in self.generators.items())
            slack_bus, slack_bus_ID = self.slack_bus, self.slack_bus.name
            cost += slack_bus.slack_cost[0] + slack_bus.slack_cost[1] * (slack_bus.baseMVA * m.P_slack[slack_bus_ID]) + slack_bus.slack_cost[2] * (slack_bus.baseMVA * m.P_slack[slack_bus_ID])**2
            return cost
        model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    def initialize_model(self):

        # Relax voltage constraints
        net.bus["max_vm_pu"] *= 1.03
        net.bus["min_vm_pu"] *= 0.97
        
        # Run power flow
        runopp(self.net, numba=False)
        
        # Extract complex bus voltages
        vm = self.net.res_bus.vm_pu  # per unit voltage magnitude
        va = np.deg2rad(self.net.res_bus.va_degree)  # voltage angle in degrees
        v = vm * np.exp(1j * va)
        v_re, v_im = v.map(np.real), v.map(np.imag)
        
        # Extract complex generator power injections
        pg = self.net.res_gen.p_mw
        qg = self.net.res_gen.q_mvar

        # Initialize values
        for k in self.model.buses:
            self.model.V_re[k].value = v_re.loc[k]
            self.model.V_im[k].value = v_im.loc[k]

        # Initialize values
        for g in self.model.generators:
            self.model.P_gen[g].value = pg.loc[g]
            self.model.Q_gen[g].value = qg.loc[g]

        # Fix slack bus voltage and voltage angle
        for slack in self.model.slack:
            self.model.V_re[slack].fix(1.0)
            self.model.V_im[slack].fix(0.0)
        
    def solve_model(self):
        solver = pyo.SolverFactory('ipopt')
        solver.solve(self.model, tee=True)

    def print_solution(self):
        # Display results
        print("\n=== Generator Dispatch (MW) ===")
        for gen in self.generators.values():
            print(f"Generator {gen.name} at bus {gen.bus.name}: P = {pyo.value(self.model.P_gen[gen.name]) * gen.baseMVA:.2f} MW, Q = {pyo.value(self.model.Q_gen[gen.name]) * gen.baseMVA:.2f} MVar")
        
        print("\n=== Bus Voltages (p.u.) ===")
        for bus in self.buses.values():
            V_re = pyo.value(self.model.V_re[bus.name])
            V_im = pyo.value(self.model.V_im[bus.name])
            V_mag = np.sqrt(V_re**2 + V_im**2)
            V_angle = np.degrees(np.arctan2(V_im, V_re))
            print(f"Bus {bus.name}: |V| = {V_mag:.4f} p.u., angle = {V_angle:.2f}°")

In [247]:
net = pn.case300()

In [248]:
model = Poly_AC_OPF(net) # Initialize model object
model.create_system() # Create power system component objects from pandapower case data
for bus in model.buses.values(): # Update bus voltage parameters
    bus.vmax *= 1.03
    bus.vmin *= 0.97
model.build_model() # Write Pyomo model constraints
model.initialize_model() # Initialize values (set starting point)
model.solve_model()
model.print_solution()

Ipopt 3.14.17: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     4604
Number of nonzeros in inequality constraint Jacobian.:      735
Number of nonzeros in Lagrangian Hessian.............:     2601

Total number of variables............................:      737
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      600
Total number

In [256]:
net.gen["vm_pu"].max()

1.0735

In [258]:
net.bus["max_vm_pu"]

0      1.06
1      1.06
2      1.06
3      1.06
4      1.06
       ... 
295    1.06
296    1.06
297    1.06
298    1.06
299    1.06
Name: max_vm_pu, Length: 300, dtype: float64

In [259]:
net = pn.case300()
net.gen["vm_pu"] = 1.0
#net.bus["max_vm_pu"] *= 1.03
#net.bus["min_vm_pu"] *= 0.97
runopp(net, numba=False)

OPFNotConverged: Optimal Power Flow did not converge!

In [234]:
# Display results
print("\n=== Generator Dispatch (MW) ===")
for gen in net.res_gen.index:
    print(f"Generator {gen} at bus {net.gen.loc[gen, 'bus']}: P = {net.res_gen.loc[gen, 'p_mw']:.2f} MW, Q = {net.res_gen.loc[gen, 'p_mw']:.2f} MVar")

print("\n=== Bus Voltages (p.u.) ===")
for bus in net.res_bus.index:
    V_mag = net.res_bus.loc[bus, "vm_pu"]
    V_angle = net.res_bus.loc[bus, "va_degree"]
    print(f"Bus {bus}: |V| = {V_mag:.4f} p.u., angle = {V_angle:.2f}°")


=== Generator Dispatch (MW) ===
Generator 0 at bus 7: P = 0.00 MW, Q = 0.00 MVar
Generator 1 at bus 9: P = 0.00 MW, Q = 0.00 MVar
Generator 2 at bus 18: P = 0.00 MW, Q = 0.00 MVar
Generator 3 at bus 54: P = 68.36 MW, Q = 68.36 MVar
Generator 4 at bus 62: P = 100.00 MW, Q = 100.00 MVar
Generator 5 at bus 68: P = 403.20 MW, Q = 403.20 MVar
Generator 6 at bus 75: P = 153.53 MW, Q = 153.53 MVar
Generator 7 at bus 76: P = 281.74 MW, Q = 281.74 MVar
Generator 8 at bus 79: P = 75.14 MW, Q = 75.14 MVar
Generator 9 at bus 87: P = 140.82 MW, Q = 140.82 MVar
Generator 10 at bus 97: P = 1973.25 MW, Q = 1973.25 MVar
Generator 11 at bus 102: P = 248.28 MW, Q = 248.28 MVar
Generator 12 at bus 103: P = 90.54 MW, Q = 90.54 MVar
Generator 13 at bus 116: P = 0.00 MW, Q = 0.00 MVar
Generator 14 at bus 119: P = 275.59 MW, Q = 275.59 MVar
Generator 15 at bus 121: P = 653.06 MW, Q = 653.06 MVar
Generator 16 at bus 124: P = 79.51 MW, Q = 79.51 MVar
Generator 17 at bus 125: P = 197.90 MW, Q = 197.90 MVar
Gene

In [249]:
net.res_cost

np.float64(720169.5718263261)

In [250]:
model.model.obj()

1034006.4370166801

In [251]:
import inspect
import pandapower

print(inspect.getsource(pandapower.run.runopp))

def runopp(net, verbose=False, calculate_voltage_angles=True, check_connectivity=True,
           suppress_warnings=True, switch_rx_ratio=2, delta=1e-10, init="flat", numba=True,
           trafo3w_losses="hv", consider_line_temperature=False, **kwargs):
    """
        Runs the  pandapower Optimal Power Flow.
        Flexibilities, constraints and cost parameters are defined in the pandapower element tables.

        Flexibilities can be defined in net.sgen / net.gen /net.load / net.storage /net.ext_grid
        net.sgen.controllable if a static generator is controllable. If False,
        the active and reactive power are assigned as in a normal power flow. If True, the following
        flexibilities apply:

        - net.gen.min_p_mw / net.gen.max_p_mw
        - net.gen.min_q_mvar / net.gen.max_q_mvar
        - net.sgen.min_p_mw / net.sgen.max_p_mw
        - net.sgen.min_q_mvar / net.sgen.max_q_mvar
        - net.dcline.max_p_mw
        - net.dcline.min_q_to_mvar / net.dcline.max_q

In [252]:
import pandapower.optimal_powerflow
print(inspect.getsource(pandapower.optimal_powerflow._optimal_powerflow))

def _optimal_powerflow(net, verbose, suppress_warnings, **kwargs):
    ac = net["_options"]["ac"]
    init = net["_options"]["init"]

    if "OPF_FLOW_LIM" not in kwargs:
        kwargs["OPF_FLOW_LIM"] = 2

    if net["_options"]["voltage_depend_loads"] and not (
            allclose(net.load.const_z_percent.values, 0) and
            allclose(net.load.const_i_percent.values, 0)):
        logger.error("pandapower optimal_powerflow does not support voltage depend loads.")

    ppopt = ppoption(VERBOSE=verbose, PF_DC=not ac, INIT=init, **kwargs)
    net["OPF_converged"] = False
    net["converged"] = False
    _add_auxiliary_elements(net)

    if not ac or net["_options"]["init_results"]:
        verify_results(net)
    else:
        init_results(net, "opf")

    ppc, ppci = _pd2ppc(net)

    if not ac:
        ppci["bus"][:, VM] = 1.0
    net["_ppc_opf"] = ppci
    if len(net.dcline) > 0:
        ppci = add_userfcn(ppci, 'formulation', _add_dcline_constraints, args=net)

    if init == 